In [1]:
import pandas as pd 
import tensorqtl
from tensorqtl import genotypeio, cis, trans

In [2]:
#----------------------------------------------------------------------
#? 基因表达量文件跑peer时，不需要header与index
#* 对基因表达量进行进一步过滤
#? 表达量大于0.5 的样本占比超过20% 
#? 存在两倍表达差异
#----------------------------------------------------------------------
geneExpression=pd.read_csv("FPKM/gene_expression.txt",header=0,index_col=0,sep="\t")
geneExpression=geneExpression.T
lowIndex=int(geneExpression.shape[0]*0.05)
highIndex=int(geneExpression.shape[0]*0.95)
fiterGenePair=[]
for gene,ratio in geneExpression.apply(lambda x: len([i for i in x if i>0.1])/geneExpression.shape[0],axis=0).iteritems():
        if ratio>=0.05:
            lowExpression=geneExpression[gene].sort_values()[lowIndex]
            highExpression=geneExpression[gene].sort_values()[highIndex]
            #* 基因在至少20%样本中表达
            if (highExpression-lowExpression)/(highExpression+lowExpression)>=0.33:
                #* 95%和5%的样本存在表达差异
                fiterGenePair.append(gene) 

fiter_expression=geneExpression[fiterGenePair]
fiter_expression.to_csv("./peer/filter_gene_expression.txt",header=False,index=False,sep=",")
pd.Series(fiterGenePair).to_csv("peer/filter_genePair.txt",header=False,index=False,sep="\t")

In [1]:
#TODO 进行eQTL mapping
import pandas as pd
import tensorqtl
from tensorqtl import genotypeio, cis, trans
import os
import torch
import sys

os.environ['CUDA_VISIBLE_DEVICES'] = "0"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('using {}'.format(device))

using cpu


In [3]:
#TODO 输入数据
plink_prefix_path ='/cotton/Liuzhenping/diaploidCotton_sQTL/Genotype/filter2_Q600_SNPs_joint_216_number_chr'
expression_bed = './phenotype/gene_expression_peer_5.bed.gz'
covariates_file = './PCA_qcovar.txt'


In [4]:
#TODO 加载数据
# load phenotypes and covariates
phenotype_df, phenotype_pos_df = tensorqtl.read_phenotype_bed(expression_bed)

covariates_df = pd.read_csv(
    covariates_file, sep='\t', index_col=0, header=None)  # samples x covariates
# * 协变量文件，最终格式为 samples x covariates
covariates_df.drop([1], axis=1)
covariates_df = covariates_df.drop([1], axis=1)
covariates_df.index.name = 'id'
covariates_df.columns = ['PC1', 'PC2', 'PC3']
covariates_df = covariates_df.loc[phenotype_df.columns]

#* 基因型文件, 对应时期的样本编号
pr = genotypeio.PlinkReader(
    plink_prefix_path, select_samples=phenotype_df.columns)

# load genotypes and variants into data frames
genotype_df = pr.load_genotypes()
variant_df = pr.bim.set_index('snp')[['chrom', 'pos']]

Mapping files: 100%|██████████| 3/3 [00:05<00:00,  1.99s/it]


In [ ]:
# 进行cis-eQTL permutate分析
cis_df = cis.map_cis(genotype_df,
                         variant_df,
                         phenotype_df,
                         phenotype_pos_df,
                         covariates_df,
                         nperm=1000,
                         maf_threshold=0.05,
                         window=1000000,
                         seed=2022
                         )

In [ ]:
# 进行cis-eQTL nominal分析
cis.map_nominal(genotype_df, variant_df,
                             phenotype_df,
                             phenotype_pos_df,
                             prefix='nominalOut',
                             covariates_df=covariates_df,
                             maf_threshold=0.05,
                             window=1000000,
                             output_dir='.',
                             write_top=True, write_stats=True)

In [7]:
#TODO 矫正pvalue 
cis_df=pd.read_csv("./All_eGene_permutate.txt",header=0,index_col=0,sep="\t")
#* 测试的基因数目太少了，修改lambda
tensorqtl.calculate_qvalues(cis_df, qvalue_lambda=0.01)

Computing q-values
  * Number of phenotypes tested: 7
  * Correlation between Beta-approximated and empirical p-values: 0.9995
  * Calculating q-values with lambda = 0.010
  * Proportion of significant phenotypes (1-pi0): 0.00
  * QTL phenotypes @ FDR 0.05: 0


In [8]:
cis_df

,num_var,beta_shape1,beta_shape2,true_df,pval_true_df,variant_id,tss_distance,ma_samples,ma_count,af,pval_nominal,slope,slope_se,pval_perm,pval_beta,qval
phenotype_id,,,,,,,,,,,,,,,,
Garb_01G000010,2910,1.060317,167.80818,154.71806,0.002920,1_360122,347814,63,117,0.297927,0.000912,0.271290,0.080528,0.373626,0.359106,0.448556
Garb_01G000020,2930,1.019808,242.78668,169.25060,0.000918,1_166471,148289,98,188,0.520619,0.000414,0.257425,0.071629,0.186813,0.192107,0.391936
Garb_01G000030,2984,1.009826,192.33210,159.18356,0.001345,1_195640,171061,12,22,0.056701,0.000428,-0.562865,0.157021,0.232767,0.223964,0.391936
Garb_01G000040,2989,1.082678,160.75177,150.88371,0.000242,1_491189,458872,22,42,0.116022,0.000034,-0.486441,0.114659,0.023976,0.028057,0.196402
Garb_01G000050,3021,1.043395,225.35234,164.27260,0.003105,1_455015,413933,102,197,0.500000,0.001386,-0.234446,0.072248,0.510490,0.483138,0.483138
Garb_01G000060,3044,0.975400,154.70288,154.78766,0.001173,1_3045,-42649,53,103,0.737245,0.000300,0.294419,0.079939,0.178821,0.175072,0.391936
Garb_01G000070,3101,1.071897,206.42955,160.35652,0.002625,1_458889,404529,18,35,0.094595,0.000992,0.420298,0.125675,0.394605,0.384476,0.448556


In [7]:
#TODO 进行trans-eQTL分析
#! 保留所有显著的位点
trans_df = trans.map_trans(genotype_df, phenotype_df, covariates_df,
                           return_sparse=True, pval_threshold=1, maf_threshold=0.05,
                           batch_size=20000)

trans-QTL mapping
  * 197 samples
  * 7 phenotypes
  * 3 covariates
  * 2143543 variants
    processing batch 108/108
    elapsed time: 0.47 min
  * 1410773 variants passed MAF >= 0.05 filtering
done.


In [10]:
trans_df=trans_df.loc[trans_df['phenotype_id']=='Garb_01G000040']

In [12]:
trans_df.to_csv("Garb_01G000040_trans_MAF.5.txt",header=True,index=False)